In [10]:
import igraph as ig
import networkx as nx
from operator import itemgetter
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D
import matplotlib.path as mpath
import time
import collections
import community
import scipy.io
from networkx.algorithms.link_analysis import hits
import itertools
#1.Creation of Graph
def createGraph(file):
    G=nx.Graph()

    mat = scipy.io.loadmat(file) #matlab file containing an adjacency matrix for the coactivation 
    #network that includes the weights of each edge and the coordinates of each node in the brain

    content = list(mat.values()) #content[3] weights of the edges; content[4] coordinates
    #print(content[3])
    G = nx.from_numpy_matrix(content[3])
    #print(G.edges[0,1]['weight'])
    return [content, G]

def averageWeightedDegree(G):
    N = G.number_of_nodes()
    weightedDegree = np.zeros(N)
    #we start by computing the weighted degree for each node (sum of weights of the edges linked to that node        
    for u, v, w in G.edges(data = 'weight'):
        #add weight to both nodes that share the edge
        weightedDegree[u] += w
        weightedDegree[v] += w
    av_weightedDegree = sum(weightedDegree)/N
    return [weightedDegree, av_weightedDegree]


file, G = createGraph('Coactivation_matrix.mat')
weightedDegree, av_w_degree = averageWeightedDegree(G)


In [11]:
Edges=list(G.edges)

Gi=ig.Graph(Edges, directed=False)


In [12]:
layt=Gi.layout('kk', dim=3) 
i=0
for node in file[4]:
    layt[i]=node
    i+=1
    

In [13]:
print(layt)

<Layout with 638 vertices and 3 dimensions>


In [14]:
print(layt[0])

[7.243636363636369, 37.01090909090908, 9.425454545454542]


In [15]:
print(file[4][0])

[ 7.24363636 37.01090909  9.42545455]


In [16]:
N=len(G.nodes)
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]  
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]  
    


In [17]:
comm_newman=[[5, 6, 10, 12, 18, 20, 37, 39, 41, 43, 46, 139, 185, 193, 213, 214, 228, 229, 233, 234, 237, 238, 239, 240, 241, 308, 309, 319, 320, 329, 354, 355, 363, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 389, 391, 392, 393, 394, 395, 396, 399, 402, 404, 405, 406, 408, 409, 410, 411, 412, 413, 414, 415, 417, 419, 420, 422, 423, 424, 452, 453, 454, 455, 462, 463, 464, 465, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 486, 487, 488, 489, 490, 495, 496, 497, 498, 500, 501, 502, 503, 504, 505, 506, 507, 510, 511, 525, 526, 527, 528, 549, 552, 553, 564, 590, 591, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631], [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 194, 195, 197, 202, 203, 204, 206, 208, 211, 212, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 300, 301, 302, 304, 305, 335, 336, 337, 338, 339, 340, 341, 342, 343, 442, 456, 457, 458, 459, 460, 461, 513, 514, 515, 521, 532, 536, 537, 538, 539, 541, 550, 551, 557, 570, 576, 580],[0, 1, 4, 8, 9, 11, 13, 14, 15, 17, 21, 22, 23, 24, 52, 54, 81, 82, 83, 84, 85, 86, 90, 99, 101, 105, 117, 134, 136, 137, 138, 141, 142, 144, 146, 147, 148, 149, 150, 151, 153, 155, 156, 157, 158, 159, 161, 162, 163, 164, 165, 166, 168, 170, 171, 175, 179, 180, 182, 183, 187, 188, 189, 190, 191, 196, 198, 199, 200, 201, 205, 207, 209, 210, 215, 216, 217, 218, 220, 221, 222, 223, 224, 225, 226, 227, 276, 294, 295, 297, 299, 306, 307, 310, 311, 312, 313, 314, 315, 316, 317, 318, 427, 428, 431, 438, 443, 445, 446, 448, 449, 451, 466, 467, 468, 469, 470, 471, 499, 508, 512, 516, 517, 518, 519, 520, 522, 523, 524, 529, 530, 531, 533, 534, 535, 540, 542, 543, 544, 545, 546, 547, 548, 554, 555, 556, 558, 559, 560, 561, 562, 563, 565, 566, 567, 568, 569, 571, 572, 573, 574, 575, 577, 578, 579, 581, 582, 583, 584, 585, 586, 587, 588, 589, 592, 593, 594, 595, 609, 632, 636, 637], [321, 322, 323, 324, 325, 326, 388, 390, 425, 426, 429, 435, 437, 439, 441],[47, 48, 49, 51, 53, 55, 56, 58, 59, 61, 72, 77, 106, 107, 108, 109, 110, 111, 112, 113, 172, 173, 174, 176, 177, 178],[2, 3, 7, 16, 19, 38, 40, 42, 44, 45, 50, 57, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 78, 79, 80, 87, 88, 89, 91, 92, 93, 94, 95, 96, 97, 98, 100, 102, 103, 104, 114, 115, 116, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 140, 143, 145, 152, 154, 160, 167, 169, 181, 184, 186, 192, 219, 230, 231, 232, 235, 236, 293, 296, 298, 303, 327, 328, 330, 331, 332, 333, 334, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 356, 357, 358, 359, 360, 361, 362, 364, 365, 397, 398, 400, 401, 403, 407, 416, 418, 421, 430, 432, 433, 434, 436, 440, 444, 447, 450, 482, 483, 484, 485, 491, 492, 493, 494, 509, 633, 634, 635]]

values=np.zeros(638)
for i in range(len(comm_newman)):
    for j in range(len(comm_newman[i])):
        values[comm_newman[i]]=i
        
values=list(values)

In [18]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='andreiarog', api_key='4QtzshXlWo5mEnzFA7zo')

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='nodes',
               marker=dict(symbol='circle',
                             size=weightedDegree*2,
                             color=values,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5), opacity=1.0
                             ),
               hoverinfo='none'
               )
trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1), opacity=0.2,
               hoverinfo='none'
               )



axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Network of brain (3D visualization)",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text="",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],    )

In [19]:
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

py.iplot(fig, file_id="brain1")

/home/andreia/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



The draw time for this plot will be slow for clients without much RAM.


/home/andreia/anaconda3/lib/python3.6/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow

